# 在Bokeh图表之间共享DataSource

In [3]:
from bokeh.models import Slider, ColumnDataSource, CustomJS
from bokeh.models.widgets import Button
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

下面的`add_source_to_window()`创建一个隐藏的`Figure`对象，在JavaScript中将`ColumnDataSource`对象的`data`属性保存到`window`全局变量之下。

In [4]:
def add_source_to_window(data, name="source_data"):
    source = ColumnDataSource(data=data)
    source.tags = [{"name":name}]
    fig = figure(plot_width=10, plot_height=10, toolbar_location=None)
    fig.line([0], [0])
    def callback_func(source=source):
        console.log(source)
        name = source.tags[0].name
        window[name] = source.data
    fig.js_on_change("inner_width", CustomJS.from_py_func(callback_func))
    fig.xaxis.visible = False
    fig.yaxis.visible = False
    fig.xgrid.visible = False
    fig.ygrid.visible = False
    fig.border_fill_color = "#ffffff"
    show(fig)

下面创建一个`DataFrame`对象，并通过`add_source_to_window()`将其保存到JavaScript的全局变量`source_data`中。

In [5]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.randn(100, 3), columns=["A", "B", "C"])
add_source_to_window(df)

下面的`fill_datasource()`为`fig`对象添加一个回调函数，当其显示时，在JavaScript中运行`callback_func()`，该函数检查全局变量`source_name`是否存在，如果存在则将`source.data`设置为该全局变量。这里通过`source.tags`在Python和JavaScript之间传递全局变量的名称。

In [9]:
def fill_datasource(fig, source, source_name="source_data"):
    source.tags = [{"source_name":source_name}]
    def callback_func(source=source):
        source_name = source.tags[0].source_name
        def change_data():
            if window.hasOwnProperty(source_name):
                source.data = window[source_name]
                source.change.emit()
            else:
                window.setTimeout(change_data, 100)
        change_data()
    fig.js_on_change("inner_width", CustomJS.from_py_func(callback_func))

在下面的例子中，创建`ColumnDataSource`对象时，只使用一行数据。在图表显示时，由JavaScript的回调函数修改其`data`属性。

In [11]:
fig = figure(plot_width=300, plot_height=300)
source1 = ColumnDataSource(df[:1])
fig.scatter(x="A", y="B", source=source1)
fill_datasource(fig, source1)
show(fig)

In [12]:
fig = figure(plot_width=300, plot_height=300)
source1 = ColumnDataSource(df[:1])
fig.scatter(x="B", y="C", source=source1)
fill_datasource(fig, source1)
show(fig)

In [16]:
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256

fig = figure(plot_width=300, plot_height=300)
source1 = ColumnDataSource(df[:1])
fig.scatter(x="A", y="B", color=linear_cmap("C", Viridis256, df.C.min(), df.C.max()), source=source1)
fill_datasource(fig, source1)
show(fig)